In [1]:
import pandas as pd
import os
import numpy as np
import scipy.stats as stats

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [2]:
PATH = 'ml/results_rf_combined_v2.csv'
PATH_REG = 'ml/results_rf_combined.csv'

In [3]:
results = pd.read_csv(PATH, index_col=0)
results_reg = pd.read_csv(PATH_REG, index_col=0)

complete = results[results.restriction == 'complete']
complete = complete.dropna(how='all', axis=1)

complete_reg = results_reg[results_reg.restriction == 'complete']
complete_reg = complete_reg.dropna(how='all', axis=1)

grouped_complete = complete.groupby(by=['restriction', 'modality', 'chunk_size'])
grouped_complete[['auc']].median()

auc
restriction modality      chunk_size          
complete    morphological 0.0         0.995410
                          25.0        0.997705
                          50.0        0.997562
                          100.0       0.997275
                          200.0       0.997418
                          400.0       0.996845
                          800.0       0.997418
                          1600.0      0.997418
            spatial       0.0         0.827883
                          1.0         0.949799
                          5.0         0.955536
                          10.0        0.962134
                          25.0        0.958692
                          50.0        0.942628
                          100.0       0.938612
                          200.0       0.928715
                          400.0       0.926277
                          800.0       0.902897
                          1600.0      0.895582
            temporal      0.0         0.975043
                          25.0        0.977194
                          50.0        0.975904
                          100.0       0.972748
                          200.0       0.975043
                          400.0       0.976908
                          800.0       0.975904
                          1600.0      0.976477

## WF

In [4]:
wf_res = complete[complete.modality == 'morphological']
wf_reg = complete_reg[complete_reg.modality == 'morphological']

chunk_sizes = wf_res.chunk_size.unique()
chunk_sizes.sort()

grouped_wf = wf_res.groupby(by=['chunk_size'])
grouped_wf[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0.0        0.25  0.978199
           0.50  0.995410
           0.75  1.000000
25.0       0.25  0.986374
           0.50  0.997705
           0.75  1.000000
50.0       0.25  0.982501
           0.50  0.997562
           0.75  1.000000
100.0      0.25  0.979920
           0.50  0.997275
           0.75  1.000000
200.0      0.25  0.985011
           0.50  0.997418
           0.75  1.000000
400.0      0.25  0.982717
           0.50  0.996845
           0.75  1.000000
800.0      0.25  0.982215
           0.50  0.997418
           0.75  1.000000
1600.0     0.25  0.980852
           0.50  0.997418
           0.75  1.000000

In [5]:
wf_aucs = [wf_res.auc[wf_res.chunk_size == cs] for cs in chunk_sizes]
wf_aucs_0 = wf_aucs[0]
print(f"comparing CS=0 to baseline for WF models:", stats.wilcoxon(wf_aucs_0-0.5, alternative='greater'))
for cs_aucs, cs in zip(wf_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for WF models:", stats.wilcoxon(cs_aucs, wf_aucs_0, alternative='greater'))
    es = cs_aucs.to_numpy() / wf_aucs_0.to_numpy()
    print(f"The effect size is: {np.quantile(es, q=[0.25, 0.5, 0.75])}")

comparing CS=0 to baseline for WF models: WilcoxonResult(statistic=1275.0, pvalue=3.130562367449191e-10)
comparing CS=0 to CS=25.0 for WF models: WilcoxonResult(statistic=455.5, pvalue=0.0034503893600137604)
The effect size is: [1.         1.         1.00461095]
comparing CS=0 to CS=50.0 for WF models: WilcoxonResult(statistic=388.0, pvalue=0.06087267929260219)
The effect size is: [1.         1.         1.00332531]
comparing CS=0 to CS=100.0 for WF models: WilcoxonResult(statistic=404.5, pvalue=0.07129639394523485)
The effect size is: [1.         1.         1.00231482]
comparing CS=0 to CS=200.0 for WF models: WilcoxonResult(statistic=443.5, pvalue=0.006272866837521202)
The effect size is: [1.         1.         1.00446686]
comparing CS=0 to CS=400.0 for WF models: WilcoxonResult(statistic=451.5, pvalue=0.012679063500033633)
The effect size is: [1.         1.         1.00285191]
comparing CS=0 to CS=800.0 for WF models: WilcoxonResult(statistic=386.5, pvalue=0.029093111331314037)
The e

largest effect size was found for 25-spikes chunks, improvement:

In [6]:
# temp here stands for tempporary not temporal
temp = complete[complete.modality == 'morphological']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_25 = temp[temp.chunk_size == 25].auc.to_numpy()
diff = 100 * (temp_25 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 25: {np.quantile(diff, q=[0.25, 0.5, 0.75])}")

Q25, Q50 and Q75  for chunk size = 25: [0.        0.        0.4610951]


In [7]:
wf_aucs_reg = [wf_reg.auc[wf_reg.chunk_size == cs] for cs in chunk_sizes]
wf_aucs_reg = wf_aucs_reg[2] # for chunk size=50
for cs_aucs, cs in zip(wf_aucs[1:], chunk_sizes[1:]):
    print(f"comparing to regular chunking with CS=50 to CS={cs} for WF models:", stats.wilcoxon(cs_aucs, wf_aucs_reg))
    es = cs_aucs.to_numpy() / wf_aucs_reg.to_numpy()
    print(f"The effect size is: {np.quantile(es, q=[0.25, 0.5, 0.75])}")

comparing to regular chunking with CS=50 to CS=25.0 for WF models: WilcoxonResult(statistic=138.5, pvalue=0.03186254722802137)
The effect size is: [0.99770214 1.         1.        ]
comparing to regular chunking with CS=50 to CS=50.0 for WF models: WilcoxonResult(statistic=127.5, pvalue=0.01068963635043735)
The effect size is: [0.99612217 1.         1.        ]
comparing to regular chunking with CS=50 to CS=100.0 for WF models: WilcoxonResult(statistic=98.5, pvalue=0.0011425229581938482)
The effect size is: [0.99646461 0.99971314 1.        ]
comparing to regular chunking with CS=50 to CS=200.0 for WF models: WilcoxonResult(statistic=194.5, pvalue=0.04835713798629708)
The effect size is: [0.99655469 0.99942628 1.        ]
comparing to regular chunking with CS=50 to CS=400.0 for WF models: WilcoxonResult(statistic=128.0, pvalue=0.006410963044012961)
The effect size is: [0.99676742 0.99913941 1.        ]
comparing to regular chunking with CS=50 to CS=800.0 for WF models: WilcoxonResult(st

## spatial

In [8]:
spat_res = complete[complete.modality == 'spatial']
spat_reg = complete_reg[complete_reg.modality == 'spatial']

chunk_sizes = spat_res.chunk_size.unique()
chunk_sizes.sort()

grouped_spat = spat_res.groupby(by=['chunk_size'])
grouped_spat[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0.0        0.25  0.802065
           0.50  0.827883
           0.75  0.848609
1.0        0.25  0.923623
           0.50  0.949799
           0.75  0.962565
5.0        0.25  0.933735
           0.50  0.955536
           0.75  0.976477
10.0       0.25  0.952811
           0.50  0.962134
           0.75  0.978055
25.0       0.25  0.947217
           0.50  0.958692
           0.75  0.966079
50.0       0.25  0.933233
           0.50  0.942628
           0.75  0.957688
100.0      0.25  0.921615
           0.50  0.938612
           0.75  0.951234
200.0      0.25  0.902539
           0.50  0.928715
           0.75  0.948365
400.0      0.25  0.906268
           0.50  0.926277
           0.75  0.943201
800.0      0.25  0.868761
           0.50  0.902897
           0.75  0.934882
1600.0     0.25  0.868330
           0.50  0.895582
           0.75  0.921974

In [9]:
spat_aucs = [spat_res.auc[spat_res.chunk_size == cs] for cs in chunk_sizes]
spat_aucs_0 = spat_aucs[0]
print(f"comparing CS=0 to baseline for Spatial models:", stats.wilcoxon(spat_aucs_0-0.5, alternative='greater'))
for cs_aucs, cs in zip(spat_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for Spatial models:", stats.wilcoxon(cs_aucs, spat_aucs_0, alternative='greater'))
    es = cs_aucs.to_numpy() / spat_aucs_0.to_numpy()
    print(f"The effect size is: {np.median(es)}")

comparing CS=0 to baseline for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7759023791403374e-10)
comparing CS=0 to CS=1.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.778464727931783e-10)
The effect size is: 1.1410076742628914
comparing CS=0 to CS=5.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.778464727931783e-10)
The effect size is: 1.1645019878834724
comparing CS=0 to CS=10.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7776104381903367e-10)
The effect size is: 1.1678936037307883
comparing CS=0 to CS=25.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7776104381903367e-10)
The effect size is: 1.1621504796565019
comparing CS=0 to CS=50.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.778464727931783e-10)
The effect size is: 1.1421589485793466
comparing CS=0 to CS=100.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7759023791403374e-10)
The effect size is: 1.1400361702413977

largest effect size (disregarding 1, 5 and 10) was found for 25-spikes chunks, improvement:

In [10]:
temp = complete[complete.modality == 'spatial']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_25 = temp[temp.chunk_size == 25].auc.to_numpy()
diff = 100 * (temp_25 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 25: {np.quantile(diff, q=[0.25, 0.5, 0.75])}")

Q25, Q50 and Q75  for chunk size = 25: [12.78780883 16.21504797 18.29261532]


largest effect size all in all was found for 10-spikes chunks, improvement:

In [11]:
temp = complete[complete.modality == 'spatial']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_10 = temp[temp.chunk_size == 10].auc.to_numpy()
diff = 100 * (temp_10 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 5: {np.quantile(diff, q=[0.25, 0.5, 0.75])}")

Q25, Q50 and Q75  for chunk size = 5: [12.27923409 16.78936037 18.94695991]


In [17]:
#  Testing if there is a peak in chunk size 10
spat_aucs = [spat_res.auc[spat_res.chunk_size == cs] for cs in chunk_sizes]
spat_aucs_10 = spat_aucs[3]
for cs_aucs, cs in zip(spat_aucs, chunk_sizes):
    if cs == 10:
        continue
    print(f"comparing CS=0 to CS={cs} for Spatial models:", stats.wilcoxon(cs_aucs, spat_aucs_10))
    es = cs_aucs.to_numpy() / spat_aucs_10.to_numpy()
    print(f"The effect size is: {np.median(es)}")

comparing CS=0 to CS=0.0 for Spatial models: WilcoxonResult(statistic=0.0, pvalue=7.555220876380673e-10)
The effect size is: 0.856243238388972
comparing CS=0 to CS=1.0 for Spatial models: WilcoxonResult(statistic=148.0, pvalue=2.29674137896291e-06)
The effect size is: 0.9833690798752335
comparing CS=0 to CS=5.0 for Spatial models: WilcoxonResult(statistic=339.0, pvalue=0.006515986735113855)
The effect size is: 0.9947399020055252
comparing CS=0 to CS=25.0 for Spatial models: WilcoxonResult(statistic=239.0, pvalue=0.00011957863254807734)
The effect size is: 0.9940773089199904
comparing CS=0 to CS=50.0 for Spatial models: WilcoxonResult(statistic=42.5, pvalue=9.25744838667388e-09)
The effect size is: 0.9834709419396196
comparing CS=0 to CS=100.0 for Spatial models: WilcoxonResult(statistic=4.0, pvalue=9.634403725281619e-10)
The effect size is: 0.9764924751152765
comparing CS=0 to CS=200.0 for Spatial models: WilcoxonResult(statistic=3.0, pvalue=9.063995250059829e-10)
The effect size is: 0

In [ ]:
spat_aucs_reg = [spat_reg.auc[spat_reg.chunk_size == cs] for cs in chunk_sizes]
spat_aucs_reg = spat_aucs_reg[4] # for chunk size=25 (note that here chunk sizes is [0, 1, 5, 10, 25, ...])
for cs_aucs, cs in zip(spat_aucs[1:], chunk_sizes[1:]):
    print(f"comparing regular chunking with CS=25 to CS={cs} for Spatial models:", stats.wilcoxon(cs_aucs, spat_aucs_reg))

## Spike-timing

In [ ]:
temp_res = complete[complete.modality == 'temporal']
temp_reg = complete_reg[complete_reg.modality == 'temporal']

chunk_sizes = temp_res.chunk_size.unique()
chunk_sizes.sort()

grouped_temp = temp_res.groupby(by=['chunk_size'])
grouped_temp[['auc']].quantile(q=[0.25, 0.5, 0.75])

In [ ]:
temp_aucs = [temp_res.auc[temp_res.chunk_size == cs] for cs in chunk_sizes]
temp_aucs_0 = temp_aucs[0]
print(f"comparing CS=0 to baseline for Temporal models:", stats.wilcoxon(temp_aucs_0 - 0.5, alternative='greater'))
for cs_aucs, cs in zip(temp_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for Spatial models:", stats.wilcoxon(cs_aucs, temp_aucs_0, alternative='greater'))
    es = cs_aucs.to_numpy() / temp_aucs_0.to_numpy()
    print(f"The effect size is: {np.median(es)}")

largest effect size was found for 1600-spikes chunks, improvement:

In [ ]:
temp = complete[complete.modality == 'temporal']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_1600 = temp[temp.chunk_size == 1600].auc.to_numpy()
diff = 100 * (temp_1600 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 800: {np.quantile(diff, q=[0.25, 0.5, 0.75])}")

In [ ]:
temp_aucs_reg = [temp_reg.auc[temp_reg.chunk_size == cs] for cs in chunk_sizes]
temp_aucs_reg = temp_aucs_reg[-1] # for chunk size=1600
for cs_aucs, cs in zip(temp_aucs[1:], chunk_sizes[1:]):
    print(f"comparing regular chunking with CS=1600 to CS={cs} for Temporal models:", stats.wilcoxon(cs_aucs, temp_aucs_reg))

## Cross-comparisons

In [ ]:
temp_1600 = complete[complete.chunk_size == 1600]
temp_1600 = temp_1600[temp_1600.modality == 'temporal'].auc

wf_25 = complete[complete.chunk_size == 25]
wf_25 = wf_25[wf_25.modality == 'morphological'].auc

spat_10 = complete[complete.chunk_size == 10]
spat_10 = spat_10[spat_10.modality == 'spatial'].auc

print('comparing WF and temporal:', stats.wilcoxon(wf_25, temp_1600))
print('comparing WF and spatial:', stats.wilcoxon(wf_25, spat_10))
print('comparing spatial and temporal:', stats.wilcoxon(spat_10, temp_1600))